# Processing of Momeni masks

In [12]:
!pip install -e /home/cerebriu/data/RESEARCH/MicrobleedNet/
import nibabel as nib
import nilearn as nil
import numpy as np 
import sys
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.spatial.distance import pdist

import cmbnet.preprocessing.loading as loading
from cmbnet.preprocessing.datasets.momeni import load_MOMENI_raw,  perform_MOMENI_QC, load_MOMENI_data

import cmbnet.visualization.utils_plotting as utils_plt
import os
from typing import Tuple, List, Dict
import matplotlib.pyplot as plt


import importlib
import cmbnet
importlib.reload(cmbnet)


Obtaining file:///home/cerebriu/data/RESEARCH/MicrobleedNet
  Preparing metadata (setup.py) ... done
  Attempting uninstall: cmbnet
    Found existing installation: cmbnet 0.1.0
    Uninstalling cmbnet-0.1.0:
      Successfully uninstalled cmbnet-0.1.0
  Running setup.py develop for cmbnet

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


<module 'cmbnet' from '/home/cerebriu/data/RESEARCH/MicrobleedNet/cmbnet/__init__.py'>

In [13]:
input_dir = "/home/cerebriu/data/datasets/raw/MOMENI"
subjects = loading.get_dataset_subjects("momeni", input_dir)

In [14]:
class Args:
    def __init__(self, input_dir = "/home/cerebriu/data/datasets/raw/MOMENI", dataset_name="momeni"):
        self.input_dir = input_dir
        self.dataset_name = dataset_name
        self.plots_path = None

In [15]:
results = {}
subjects_temp = subjects
subjects_temp = ['2_T2_MRI_SWI_BFC_50mm_HM']
for subject in subjects_temp:
    print("------------------------------------------------")
    print(subject)
    print("------------------------------------------------")

    # Load data
    args = Args()
    sequences_raw, labels_raw, sequence_type, com_list = load_MOMENI_raw(args.input_dir, subject)
    sequences_qc, labels_qc, labels_metadata, msg = perform_MOMENI_QC(subject, sequences_raw, labels_raw, com_list, "")

    mri_im_raw = sequences_raw[sequence_type]
    rawcmb_im_raw = labels_raw[sequence_type]
    grownmask_raw = labels_qc[sequence_type]

    results[subject] = {
        "raw": (sequences_raw, labels_raw, sequence_type, com_list),
        "qc": (sequences_qc, labels_qc, labels_metadata, msg)
    }
    print(f"CMBs: {com_list}")
    print(msg)

    print(f"shape: {mri_im_raw.shape}")

    # Plotting processed masks for each center of mass
    for i, CM in enumerate(com_list):
        filename_temp = os.path.join(args.input_dir, f"MOMENI_{subject}.png")                                
        metadata_str = f"""\
        sub: {subject}
        {sequence_type}, shape: {mri_im_raw.shape}
        CMBloc: {CM}   CMBsize: {labels_metadata[sequence_type]["CMBs_old"][i]['size']}
        "connectivity": {labels_metadata[sequence_type]["CMBs_old"][i]['region_growing']['connectivity']}, "intensity_mode": {labels_metadata[sequence_type]["CMBs_old"][i]['region_growing']['intensity_mode']}, 
        "diff_mode": {labels_metadata[sequence_type]["CMBs_old"][i]['region_growing']['diff_mode']}, 
        "distance_th": {labels_metadata[sequence_type]["CMBs_old"][i]['region_growing']['distance_th']},"size_th": {labels_metadata[sequence_type]["CMBs_old"][i]['region_growing']['size_th']}"""
        utils_plt.plot_processed_mask(mri_im_raw, rawcmb_im_raw, grownmask_raw, CM, 100, metadata_str=metadata_str, save_path=filename_temp)

------------------------------------------------
2_T2_MRI_SWI_BFC_50mm_HM
------------------------------------------------
CMBs: []
		Thresholds for RegionGrowing --> Max. distance =5, Max Size=340
		Processing CMB annotations 

shape: (176, 256, 80)


In [16]:
labels_metadata

{'SWI': {'healthy': 'yes', 'CMBs_old': {}}}

In [17]:
sequences_qc, labels_qc, labels_metadata, sequence_type, msg = load_MOMENI_data(args, subject, "")